![](../images/dtlogo.png)

In [ ]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython import display
import cv2
import copy
from tqdm.contrib import tzip

# Particle filter

Robots use sensors to estimate its state. However, as we know, sensors are noisy and thus should not be fully trusted. The particle filter allows us to estimate state by tracking some set of likely states and updating these as we get new measurements. This is achieved using a set of particles, where each particle $i$ represents a hypothesis of the state state $x_t^i$ with its associated weight $w_t^i$. So for example, in the context of state estimation (e.g., localization), each particle represents how likely the robot is located at a particular state.

The algorithm is as follows. First, we initialize the particle filter by sampling $M$ particles. Usually the initial set of particles can be sampled uniformly from the set of all possible states (i.e., uniform sampling). Given a control signal $u_t$, we move all the particles (i.e., the state associated with each particle) that we currently have following the motion model. After this, we take a measurement $z_t$ (which we can get from our sensor) and update the weights of each particle based on how likely we are to measure $z_t$ if we were at the corresponding state of each particle (e.g., $p(z_t | x_i)$). We then resample all the particles, not uniformly, but according to the weights of each particle (i.e., the ones with higher weights are more likely to be sampled). This means that the particles that have higher weights will be sampled more often, while the ones with low weights may not be sampled at all. There are different options to obtain the predicted (i.e., filtered) state. In this notebook, we will do the simplest approach by taking the mean of the resampled particles.

We then repeat this process everytime we have new $u_t$ and $z_t$. In practice, we may not know what $p(z_t|x_i)$ is, so we may need other ways to compute the particle weight. We will see this later in the implementation.

The above algorithm is the basic algorithm for the particle filter. There are some associated challenges and different algorithm variations try to address these issues. For example, if we implement the above algorithm, the performance will be largely affected by the initial set of particles because during resampling step, we only sample particles from the set of initial particles. Thus, it is possible that we may not even have particles that are near the correct state. This problem is sometimes known as particle depletion. To help alleviate this problem a little bit, during resampling step, instead of sampling completely from the set of particles that we currently have according to their weights, we can sample a small percentage of particles randomly from the set of all possible particles. In this notebook, we will stick with this approach since it is the simplest to implement and allows us to understand the basics of particle filter.

Let's take a look at a concrete example.

**Example: estimating robot position with particle filter**

Consider a robot moving in a room without obstacles. The robot is equipped with two sensors to measure distance between the robot and the walls, which allows the robot to measure the location of the robot (i.e., $x$ and $y$ positions) in the room. For the sake of simplicity, assume that the sensors directly provide us with a noisy measurement of $(x,y)$ location in the room.

Say the state of the robot is its $x$ and $y$ position in the room, and the control inputs are the velocity in each direction $v_x$ and $v_y$. The robot is initialized at $(x,y) = (0,0)$, and moves by applying constant control inputs $v_x = v_y = 0.1$ for 100 time steps. At each time step, after applying a control signal, the robot can take a measurement using the sensors to have an idea where it currently is. 

Given the motion model:

$$
x_t = x_{t-1} + u_t + w_t,
$$

where $w_t \sim \mathcal{N}(0,0.01)$

we will use the particle filter to improve our estimate of where the robot is at each time step.

**Understanding the problem**

To understand the problem, let us plot the ideal trajectory and some possible measurements that we would get using the available sensors according to their specifications. In real life, we can get sensor measurements from our sensors. But here, to simulate sensor measurements, we will take the ground truth state at every time step and add some Gaussian noise from $\mathcal{N}(0, 0.5)$.

In [ ]:
# state = [x_pos, y_pos]
num_data = 100
ground_truth_x = np.linspace(0, 10, num=num_data + 1)
ground_truth_y = ground_truth_x.copy() # x = y
    
# Simulate sensor measurements
measurement_noise_x_var = 0.5
measurement_noise_y_var = 0.5
noise_x = np.random.normal(loc=0.0, scale=measurement_noise_x_var, size=num_data-1)
noise_y = np.random.normal(loc=0.0, scale=measurement_noise_y_var, size=num_data-1)
measurement_x = np.linspace(10 / num_data, 10, num=num_data-1) + noise_x
measurement_y = np.linspace(10 / num_data, 10, num=num_data-1) + noise_y

# Compare ground truth and measurements
plt.figure(figsize=(8,8))
plt.plot(ground_truth_x, ground_truth_y)
plt.plot(measurement_x, measurement_y)
plt.xlabel('x position')
plt.ylabel('y position')
plt.legend(['ground truth trajectory', 'localization measurements from noisy sensor'])
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

As we can see there is quite a lot of noise coming from the sensor.

**Implementing particle filter**

First we create a Particle class. Each particle has its state and weight. It also has a method to move itself given a control signal $u_t$ (i.e., predict step) and a method to update its own weight given a measurement $z_t$. Since we do not know what $p(z_t|s_i)$ is, we compute the weight of each particle based on how far it is from the measurement. Intuitively, the larger the distance between $s_i$ and $z_t$ is, the lower the probability for the robot to measure $z_t$ if the it was at $s_i$. Note that this is only a simple option to approximate $p(z_t|s_i)$, and may not work too well!

In [ ]:
class Particle():
    def __init__(self, x, y, w):
        self.state = np.array([x, y])
        self.weight = w
    
    def predict(self, u_t):
        motion_model_noise = np.random.normal(0, 0.01, size=self.state.shape)
        self.state = self.state + u_t + motion_model_noise
        
    def update(self, z_t):
        dist_from_state_to_z = np.linalg.norm(self.state - z_t)
        self.weight = 1. / dist_from_state_to_z

Next, we implement the particle filter and run it for each time step.

In [ ]:
num_particles = 1000 # pick a number

filtered_xs = []
filtered_ys = []
measurements = []
particles_over_time = []

# sample initial particles
particles = []
for i in range(num_particles):
    x = np.random.uniform(0, 10)
    y = np.random.uniform(0, 10)
    w = 1
    particles.append(Particle(x, y, w))

# we assume constant control signal in this particular example
u_t = np.array([10./num_data, 10./num_data]) 
    
# run particle filter for each time step
for i in range(num_data):
    
    # given u_t, move all the particles following the motion model
    for p in particles:
        p.predict(u_t)
    
    # get measurement z_t (in real life, get this from our sensor instead)
    measurement_noise_x = np.random.normal(loc=0.0, scale=measurement_noise_x_var)
    measurement_noise_y = np.random.normal(loc=0.0, scale=measurement_noise_y_var)
    measurement_x_new = ground_truth_x[i+1] + measurement_noise_x
    measurement_y_new = ground_truth_x[i+1] + measurement_noise_y
    z_t = np.array([measurement_x_new, measurement_y_new])
    measurements.append([measurement_x_new, measurement_y_new])
    
    # given z_t, update particles' weights
    for p in particles:
        p.update(z_t)
    
    # resample particles using the updated weights
    alpha = 0.9
    weights = [] 
    for p in particles:
        weights.append(p.weight)
    normalized_weights = np.array(weights) / float(np.sum(weights)) # array of normalized weights from each particle
    indexes = np.random.choice(num_particles,
                               size=num_particles,
                               p=normalized_weights)
    for j in range(int(num_particles * alpha)):
        p = particles[indexes[j]]
        particles[j] = p
    for j in range(num_particles - int(num_particles * alpha)):
        x = np.random.uniform(0,10)
        y = np.random.uniform(0,10)
        w = 0.
        particles[int(num_particles * alpha) + j] = Particle(x,y,w)
        
    # get state estimate by taking the mean of the resampled particles (excluding the randomly sampled ones)
    xs = []
    ys = []
    for p in particles[:int(num_particles * alpha)]:
        xs.append(p.state[0])
        ys.append(p.state[1])
    estimated_x = np.mean(xs)
    estimated_y = np.mean(ys)
    
    # store filtered state so we can plot them later
    filtered_xs.append(estimated_x)
    filtered_ys.append(estimated_y)
    
    # store resampled particles so we can plot them later
    particles_over_time.append(copy.deepcopy(particles))
measurements = np.array(measurements)

Let's visualize how the particles evolve over time so we can get a sense on how particle filter works.

In [ ]:
ground_truth = np.stack((ground_truth_x, ground_truth_y), axis=1)
filtered_states = np.stack((filtered_xs, filtered_ys), axis=1)

In [ ]:
plots = []

t = 0
for (set_of_ps, true_state, filtered_state, z_t) in tzip(particles_over_time, ground_truth[1:], filtered_states, measurements):
    fig = plt.figure()
    plt.scatter(true_state[0], true_state[1], color='blue', s=75)
    plt.scatter(filtered_state[0], filtered_state[1], color='green', s=75)
    plt.scatter(z_t[0], z_t[1], color='orange', s=75)
    
    for p in set_of_ps:
        plt.scatter(p.state[0], p.state[1], color='red', s=p.weight * 25)
    plt.title('time=%d, true (x,y)=(%.2f, %.2f) \n filtered (x,y)=(%.2f, %.2f)' 
              % (t+1, true_state[0], true_state[1], filtered_state[0], filtered_state[1]))
    plt.xlabel('x position')
    plt.ylabel('y position')
    plt.xlim(0,15)
    plt.ylim(0,15)
    plt.gca().set_aspect('equal', adjustable='box')
    
    blue_patch = patches.Patch(color='blue', label='true state')
    red_patch = patches.Patch(color='red', label='weighted particles')
    green_patch = patches.Patch(color='green', label='filtered state')
    orange_patch = patches.Patch(color='orange', label='noisy measurement')
    plt.legend(handles=[blue_patch, red_patch, green_patch, orange_patch], loc='upper left', fontsize=8)
    
    fig.canvas.draw()
    plot_img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    plot_img = plot_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plots.append(plot_img)
    t += 1
    plt.clf()
    plt.close()

In [ ]:
for im in plots:
    plt.figure(figsize=(14,14))
    plt.imshow(im)
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(0.01)
    plt.clf()
    plt.close()

In [ ]:
# Run this if we want to save the plot animation as gif

# import imageio
# imageio.mimsave('pf.gif', plots)

As we see, the particles (i.e., red dots) that are closer to where the robot is (i.e., blue dot) are getting bigger over time (i.e., gain more weights) as the robot moves. Here, the state predicted by the particle filter is visualized as the green dot, while the sensor measurement is visualized as the orange dot.

Notice that it takes some time for the particles to "converge" to the true state. This can be observed by looking at the poor performance of state estimation at earlier time steps. This is because initially the particles were spawned randomly so it will not produce a good estimation until we have enough measurements. Overall, the filter seems to work although not too well, let's plot the state estimation over time below.

In [ ]:
# Let's plot the results
plt.figure(figsize=(8,8))
plt.plot(ground_truth[:,0], ground_truth[:,1])
plt.plot(measurements[:,0], measurements[:,1])
plt.plot(filtered_states[:,0], filtered_states[:,1])
plt.xlabel('x position')
plt.ylabel('y position')
plt.legend(['ground truth trajectory', 'noisy measurements', 'particle filter estimate'])
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

The filter seems to work as the green lines is close to the $x=y$ line, but not too good. What can we do to improve the performance?

While having more particles corresponds to better approximation of the true distribution that we are trying to estimate, it makes computation to be more expensive, so this can be a limitation depending on the hardware used. 

Try experimenting with some of the parameters to see if you can improve things further. 